In [1]:
import pandas as pd

In [2]:
df_train_tweets=pd.read_csv('Train.csv')
df_test_tweets=pd.read_csv('Test.csv')

In [3]:
df_train_tweets.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [4]:
df_train_tweets.safe_text[4]

'Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>'

In [5]:
df_train_tweets.drop(['agreement','tweet_id'],inplace=True,axis=1)

In [6]:
df_train_tweets.head()

,safe_text,label
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0
1,I'm 100% thinking of devoting my career to pro...,1.0
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0
3,I mean if they immunize my kid with something ...,-1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0


In [7]:
df_train_positif=df_train_tweets[df_train_tweets['label']==1.0].reset_index()
df_train_neutral=df_train_tweets[df_train_tweets['label']==0.0].reset_index()
df_train_negatif=df_train_tweets[df_train_tweets['label']==-1.0].reset_index()

In [8]:
print(df_train_positif.head())

   index                                          safe_text  label
0      1  I'm 100% thinking of devoting my career to pro...    1.0
1      5  <user> a nearly 67 year old study when mental ...    1.0
2      6  Study of more than 95,000 kids finds no link b...    1.0
3      7                   psa: VACCINATE YOUR FUCKING KIDS    1.0
4      8  Coughing extra on the shuttle and everyone thi...    1.0


In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

positive_token=[]
negative_token=[]
neutral_token=[]
test_token=[]

In [10]:
df_test_tweets.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [11]:
for i in  range(len(df_train_positif)):
    positive_token.append(word_tokenize(df_train_positif.safe_text[i]))
for j in  range(len(df_train_negatif)):
    negative_token.append(word_tokenize(df_train_negatif.safe_text[j]))    
for t in  range(len(df_train_neutral)):
    neutral_token.append(word_tokenize(df_train_neutral.safe_text[t]))  
for k in  range(len(df_test_tweets)):
    test_token.append(word_tokenize(str(df_test_tweets.safe_text[k])))    

In [41]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, MaxentClassifier ,NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token=re.sub("'s'","",token)
        token=re.sub("user","",token)
        token=re.sub("url","",token)
        token=re.sub('😑',"",token)     
        """"
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        """ 
        #lemmatizer = WordNetLemmatizer()
        #token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [22]:
stop_words = stopwords.words('english')
stop_words.append(['user','url','``','..','...'])
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
neutral_cleaned_tokens_list= []
test_cleaned_tokens_list=[]

In [23]:
for tokens in positive_token:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_token:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in neutral_token:
        neutral_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in test_token:
        test_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [42]:
print(positive_cleaned_tokens_list)

[["'m", '100', 'thinking', 'devoting', 'career', 'proving', 'autism', "n't", 'caused', 'vaccines', 'due', 'idiotic', 'posts', "'ve", 'seen', 'world', 'autism', 'day'], ['nearly', '67', 'year', 'old', 'study', 'mental', 'health', 'studies', 'vaccines', 'relatively', 'infancies', 'refuted'], ['study', '95,000', 'kids', 'finds', 'link', 'mmr', 'vaccine', 'autism'], ['psa', 'vaccinate', 'fucking', 'kids'], ['coughing', 'extra', 'shuttle', 'everyone', 'thinks', 'measles', '😂', 'vaccinateyourkids'], ['aids', 'vaccine', 'created', 'oregon', 'health', 'amp', 'science', 'university', 'may', 'clear', 'virus', 'body'], ['getting', 'vaccines', 'china', 'nervous', 'moving', '..', 'cheryl', 'southern', 'nevada', 'health', 'district'], ['got', 'influenza', 'vaccine', 'purdue', 'university', 'student', 'health', 'push'], ['thank', 'standing', 'ca', 'parents', 'children', 'support', 'vaccineswork'], ['beeftalk', 'start', 'calf', 'vaccinations', 'via', 'good', 'article', 'herd', 'health'], ["n't", 'care

In [92]:

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
neutral_tokens_for_model = get_tweets_for_model(neutral_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

neutral_dataset = [(tweet_dict, "Neutral")
                         for tweet_dict in neutral_tokens_for_model]

dataset = positive_dataset + negative_dataset + neutral_dataset

random.shuffle(dataset)

train_data = dataset[:9500]
test_data = dataset[9500:]
classifier = NaiveBayesClassifier.train(train_data)   
#classifier = MaxentClassifier.train(train_data, max_iter=50,min_accdelta=0.05)
print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(20))


Accuracy is: 0.6332665330661322
Most Informative Features
                 average = True           Neutra : Positi =     73.1 : 1.0
        cdcwhistleblower = True           Negati : Positi =     67.5 : 1.0
               increases = True           Neutra : Positi =     55.7 : 1.0
                complain = True           Neutra : Positi =     54.4 : 1.0
                 health. = True           Neutra : Positi =     49.9 : 1.0
                  boosts = True           Neutra : Positi =     45.8 : 1.0
                vaxtruth = True           Negati : Neutra =     36.2 : 1.0
       vaccinateyourkids = True           Positi : Neutra =     33.5 : 1.0
                  admits = True           Negati : Neutra =     33.1 : 1.0
                aluminum = True           Negati : Neutra =     29.9 : 1.0
                      dc = True           Neutra : Positi =     28.9 : 1.0
                  causes = True           Negati : Neutra =     23.6 : 1.0
               paralyzed = True           

In [93]:
print("train accuracy=", classify.accuracy(classifier, train_data))
print("test accuracy=", classify.accuracy(classifier, test_data))

train accuracy= 0.7866315789473685
test accuracy= 0.6332665330661322


In [77]:
len(positive_dataset+negative_dataset)

5091

In [78]:
df_submission=pd.read_csv('SampleSubmission.csv')

In [79]:
predicted=[]
i=0
for custom_tweet in test_cleaned_tokens_list:
    
    custom_tokens = remove_noise(custom_tweet)
    #print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))
    if(classifier.classify(dict([token, True] for token in custom_tokens))=="Positive"):
        predicted.append(1.0)
    elif(classifier.classify(dict([token, True] for token in custom_tokens))=="Negative"):
        predicted.append(-1.0)
    else:
        predicted.append(0.0)

In [80]:
pd.set_option('mode.chained_assignment', None)

In [81]:
for i in range(len(predicted)):
    df_submission.label[i]=float(predicted[i])

In [82]:
print(df_submission.label.count())
print(df_submission.tweet_id.count())
print(df_test_tweets.tweet_id.count())
df_submission[df_submission.tweet_id=='01AXPTJF']

5177
5177
5177


,tweet_id,label
2,01AXPTJF,0


In [83]:
df_submission.set_index('tweet_id',inplace=True)

In [84]:
df_submission.to_csv('submission2.csv')

In [85]:
df_submission

,label
tweet_id,
00BHHHP1,1
00UNMD0E,1
01AXPTJF,0
01HOEQJW,1
01JUKMAO,0
01V1X8XW,0
01VCVYR0,0
01VO7KBO,1
020SYHIL,0
